In [ ]:
import pandas as pd

### krx 사이트에 등록되어 있는 상장사 정보(html) 다운로드 받아 dataframe 객체 생성
* krx 상장사 정보 url : http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13

In [ ]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

In [ ]:
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


### Q1. 상장사 정보 중 종목코드를 왼쪽이 '0' 으로 채워진 6자리 값으로 변환하시오

In [ ]:
code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)

### Q2. code_df 객체에서 '회사명', '종목코드' 컬러만 선택하여 code_df 객체에 재 할당 하시오.

In [ ]:
code_df = code_df[['회사명', '종목코드']]

In [ ]:
code_df.head()

,회사명,종목코드
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


### Q3. code_df 객체에서 '회사명' , '종목코드' 컬럼의 이름을 각 각 'name', 'code' 로 변경 하시오

In [ ]:
code_df = code_df.rename(columns={'회사명' : 'name',
                                  '종목코드' : 'code'})

In [ ]:
code_df.head()

,name,code
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


### Q4. get_url 함수를 정의하시오
* 입럭받은 '회사명'(item_name) 정보와 code_df 객체를 이용하여 '상장 코드' 를 이용하여 naver 주식 시세 정보를 검색하는 url을 작성하시오
* naver 주식 시세 정보 검색 url : https://finance.naver.com/item/sise_day.nhn?code=`상장 코드`
* 작성한 함수를 이용하여 삼성전자 주식 조회를 위한 naver 주식 시세 정보 검색 url 정보를 출력하시오

In [ ]:
def get_url(item_name, code_df):
  code = code_df['code'][code_df['name'] == item_name].values[0]
  url = 'https://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)

  print("request URL = {}".format(url))
  return url

In [ ]:
item_name='삼성전자'
url = get_url(item_name, code_df)

request URL = https://finance.naver.com/item/sise_day.nhn?code=005930


### url을 이용한 주식시세 정보 호출 및 html 파싱을 위한 bs4 라이브러리 임포트

In [ ]:
import requests
from bs4 import BeautifulSoup

### Q5. Q4에서 생성한 naver 주식 시세정보 조회 url을 이용하여 200일 간의 삼성전자의 주식 시세 정보를 리스트 객체에 저장하시오
* list 의 shape = (200(일수), 7(날짜, 종가, 차액, 시가, 상한, 하한, 거래량))

In [ ]:
df = pd.DataFrame()

priceList = []

for page in range(1, 21):
  pg_url = '{url}&page={page}'.format(url=url, page=page)
  
  response= requests.get(pg_url)
  soup = BeautifulSoup(response.content, 'lxml')

  trList = soup.body.find_all(name='tr', attrs={'onmouseover':'mouseOver(this)'})
  
  for tr in trList:
    priceList.append([td.span.string.strip() for td in tr.find_all(name='td')])

### Q6. priceList 정보를 이용하여 df 이름을 dataframe 객체 생성
* 컬럼의 이름은 'date', 'close', 'diff', 'open', 'high', 'low', 'volume' 롤 지정

In [ ]:
df = pd.DataFrame(priceList,
                  columns=['date', 'close', 'diff', 'open', 'high', 'low', 'volume']
                  )

### Q7. df 객체에서 Nan 제거

In [ ]:
df = df.dropna()
df.head()

,date,close,diff,open,high,low,volume
0,2020.05.22,"48,750","1,200","49,600","49,800","48,600","19,289,745"
1,2020.05.21,"49,950",50,"50,300","50,400","49,850","14,949,266"
2,2020.05.20,"50,000",300,"50,000","50,200","49,800","14,896,899"
3,2020.05.19,"50,300","1,500","50,100","50,500","49,700","25,168,295"
4,2020.05.18,"48,800",950,"47,950","49,100","47,600","20,481,981"


### df 객체에 각 원소들의 값이 object로 되어 있기 때문에 분석을 위해 float type으로 

In [ ]:
df.dtypes

date      object
close     object
diff      object
open      object
high      object
low       object
volume    object
dtype: object

In [ ]:
import locale
from locale import atof

locale.setlocale(locale.LC_NUMERIC, '')

df['date'] = pd.to_datetime(df['date'])

df[['close', 'diff', 'open', 'high', 'low', 'volume']] = \
df[['close', 'diff', 'open', 'high', 'low', 'volume']].applymap(atof)
df = df.sort_values(by=['date'], ascending=True)

In [ ]:
df.dtypes

date      datetime64[ns]
close            float64
diff             float64
open             float64
high             float64
low              float64
volume           float64
dtype: object

In [ ]:
df.head()

,date,close,diff,open,high,low,volume
199,2019-07-30,46550.0,450.0,46300.0,46850.0,46300.0,5551014.0
198,2019-07-31,45350.0,1200.0,46200.0,46600.0,45000.0,12872916.0
197,2019-08-01,45200.0,150.0,44900.0,45500.0,44850.0,7811181.0
196,2019-08-02,44950.0,250.0,44550.0,45500.0,44300.0,12151374.0
195,2019-08-05,43950.0,1000.0,44350.0,44600.0,43600.0,14214086.0


### 삼성 전자 종가 기준 일별 주가 시계열 차트 조회

In [ ]:
# 시계열 차트 생성을 위한 라이브러리 임포트
import plotly.graph_objects as go

In [ ]:
# 차트 생성을 위한 데이터 생성
trace = go.Scatter(x=df['date'],
                   y=df['close'],
                   name=item_name)

data = [trace]

In [ ]:
# 시계열 차트의 레이웃 정보 생성
layout = dict(
    title='{}의 종가(close) Time Series'.format(item_name),
    # x축 데이터 타입 지정(date)
    xaxis = dict(
        rangeslider=dict(),
        type='date'
        ),
)

In [ ]:
# 차트 객체 생성 및 조회
fig = go.Figure(data=data, layout=layout)
fig.show()